# Snowpark For Python -- Titanic Survival Prediction

### In this session, we will cover:

* Snowpark for Python Installation
* Creating Session object and connecting to Snowflake
* Loading data from Snowflake table into Snowpark DataFrame
* Creating Stored Procedure to deploy model training code on Snowflake
* Creating User-Defined Function (UDF) for inference

### Import libraries

In [3]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark import Column
from snowflake.snowpark import Window

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [5]:
import json

import pandas as pd

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Connect to Snowflake

In [6]:
with open('../creds.json') as f:
    connection_parameters = json.load(f)

In [7]:
session = Session.builder.configs(connection_parameters).create()

In [8]:
print(f"Current schema: {session.get_fully_qualified_current_schema()}, current role: {session.get_current_role()}, current warehouse:  {session.get_current_warehouse()}")


Current schema: "DEMO_DB"."PUBLIC", current role: "SYSADMIN", current warehouse:  "COMPUTE_WH"


### Create a DataFrame based on the Titanic table

In [9]:
titanic_df = session.table("titanic")

In [10]:
print(f"Number of rows: {titanic_df.count()}")
titanic_df.show()

Number of rows: 1309
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PCLASS"  |"SURVIVED"  |"NAME"                                           |"SEX"   |"AGE"   |"SIBSP"  |"PARCH"  |"TICKET"  |"FARE"    |"CABIN"  |"EMBARKED"  |"BOAT"  |"BODY"  |"HOME_DEST"                      |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |1           |Allen, Miss. Elisabeth Walton                    |female  |29.0    |0.0      |0.0      |24160     |211.3375  |B5       |S           |2       |NULL    |St Louis, MO                     |
|1.0       |1           |Allison, Master. Hudson Trevor                   |male    |0.9167  |1.0      |2.0      |113781    |151.55    |

In [11]:
titanic_df.queries

{'queries': ['SELECT  *  FROM (titanic)'], 'post_actions': []}

### Basic analysis


Count by Survived

In [12]:
titanic_df.group_by("SURVIVED").count().show()

------------------------
|"SURVIVED"  |"COUNT"  |
------------------------
|1           |500      |
|0           |809      |
------------------------



Add percentages

In [13]:
titanic_df.group_by(F.col("SURVIVED")).agg(F.count('*').as_("PASSENGERS"))\
            .select(F.col("SURVIVED"), F.col("PASSENGERS"), (F.call_builtin("RATIO_TO_REPORT", F.col("PASSENGERS")).over() * 100).as_("percentage") )\
            .show()

--------------------------------------------
|"SURVIVED"  |"PASSENGERS"  |"PERCENTAGE"  |
--------------------------------------------
|1           |500           |38.197100     |
|0           |809           |61.802900     |
--------------------------------------------



Describe all numeric and categorical feature

In [14]:
titanic_df.describe().show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"PCLASS"            |"SURVIVED"  |"NAME"                       |"SEX"   |"AGE"               |"SIBSP"             |"PARCH"             |"TICKET"   |"FARE"             |"CABIN"  |"EMBARKED"  |"BOAT"  |"BODY"             |"HOME_DEST"          |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |1309.0              |1309        |1309                         |1309    |1046.0              |1309.0              |1309.0              |1309       |1308.0             |295      |1307        |486     |121.0              

Based on above statistics can drop some of the columns

In [15]:
titanic_df = titanic_df.drop(["NAME", "TICKET", "CABIN", "BOAT", "BODY", "HOME_DEST"])
titanic_df.show()

---------------------------------------------------------------------------------------
|"PCLASS"  |"SURVIVED"  |"SEX"   |"AGE"   |"SIBSP"  |"PARCH"  |"FARE"    |"EMBARKED"  |
---------------------------------------------------------------------------------------
|1.0       |1           |female  |29.0    |0.0      |0.0      |211.3375  |S           |
|1.0       |1           |male    |0.9167  |1.0      |2.0      |151.55    |S           |
|1.0       |0           |female  |2.0     |1.0      |2.0      |151.55    |S           |
|1.0       |0           |male    |30.0    |1.0      |2.0      |151.55    |S           |
|1.0       |0           |female  |25.0    |1.0      |2.0      |151.55    |S           |
|1.0       |1           |male    |48.0    |0.0      |0.0      |26.55     |S           |
|1.0       |1           |female  |63.0    |1.0      |0.0      |77.9583   |S           |
|1.0       |0           |male    |39.0    |0.0      |0.0      |0.0       |S           |
|1.0       |1           |female 

Let's have a look at the datatypes for the remaining colums

In [16]:
for col in titanic_df.schema.fields:
    print(f"{col.name}, Nullable: {col.nullable}, {col.datatype}")

PCLASS, Nullable: True, DoubleType()
SURVIVED, Nullable: True, StringType()
SEX, Nullable: True, StringType()
AGE, Nullable: True, DoubleType()
SIBSP, Nullable: True, DoubleType()
PARCH, Nullable: True, DoubleType()
FARE, Nullable: True, DoubleType()
EMBARKED, Nullable: True, StringType()


PCLASS is stored as a number but is a category variable so we can change it character instead

In [17]:
titanic_df = titanic_df.with_column("PCLASS", F.to_varchar("PCLASS"))

In [18]:
titanic_df.group_by("EMBARKED").count().show()

------------------------
|"EMBARKED"  |"COUNT"  |
------------------------
|S           |914      |
|C           |270      |
|NULL        |2        |
|Q           |123      |
------------------------



In [19]:
titanic_df = titanic_df.fillna({"EMBARKED": "S"})
titanic_df.group_by("EMBARKED").count().show()

------------------------
|"EMBARKED"  |"COUNT"  |
------------------------
|S           |916      |
|C           |270      |
|Q           |123      |
------------------------



Next let’s look at the relationship between each of the features and our target variable. We can reuse the crosstab function craeted earlier for that

In [20]:
cols = [c.name for c in titanic_df.schema.fields if type(c.datatype) == T.StringType]
for col in cols:
    if col != "SURVIVED":
        window = Window.partition_by(col)
        display(titanic_df.group_by(col, "SURVIVED").count().select(col, "SURVIVED", (F.call_builtin("RATIO_TO_REPORT", F.col("COUNT")).over(window) * 100).as_("percentage") ).pivot("SURVIVED", ['0', '1']).agg(F.sum("percentage")).show(20))


----------------------------------
|"SEX"   |"'0'"      |"'1'"      |
----------------------------------
|female  |27.253200  |72.746800  |
|male    |80.901500  |19.098500  |
----------------------------------



None

--------------------------------------
|"EMBARKED"  |"'0'"      |"'1'"      |
--------------------------------------
|S           |66.593900  |33.406100  |
|C           |44.444400  |55.555600  |
|Q           |64.227600  |35.772400  |
--------------------------------------



None

------------------------------------
|"PCLASS"  |"'0'"      |"'1'"      |
------------------------------------
|1         |38.080500  |61.919500  |
|2         |57.039700  |42.960300  |
|3         |74.471100  |25.528900  |
------------------------------------



None

### Model training

We are going to create a function for training a model that we will run in Snowflake as a Stored Procedure.

Start with selecting the columns we are going to use and pull the data back as a Pandas dataframe so we can test the function locally, if we had have lota of data we would have taken a sample.

In [21]:
df = titanic_df.sample(frac=0.10).to_pandas()

X = df[["EMBARKED", "SEX", "PCLASS", "AGE", "FARE"]]
y = df["SURVIVED"]

Define the training function where we will also do some data preprocessing, by using Pipeline we can then reuse the processing when using the model for prediictions.

In [24]:
def train(X, y):
    cat_cols = ["EMBARKED", "SEX", "PCLASS"]
    cat_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    
    num_cols = ["AGE", "FARE"]
    num_transformer = Pipeline(steps=[
        ('imputer', KNNImputer(n_neighbors=5)),
        ('scaler', RobustScaler())
    ])
    preprocessor = ColumnTransformer(
      [
            ('num', num_transformer, num_cols),
            ('cat', cat_transformer, cat_cols)
        ],  verbose_feature_names_out=False,
    )
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])
    model = pipe.fit(X, y)
    return model


Test the function locally.

In [25]:
model = train(X, y)

In [31]:
def create_udf(model, session):
    @F.udf(name = 'survived', is_permanent = True, stage_location = 'udf_stage' , packages = ['pandas', 'scikit-learn'], replace = True, session = session)
    def survived(ds: T.PandasSeries[dict]) -> T.PandasSeries[float]:
        df = pd.io.json.json_normalize(ds)
        prediction = model.predict(df)
        return prediction


In [32]:
def train_titanic(session: Session) -> str:
    df = session.table("titanic").select("EMBARKED", "SEX", "PCLASS", "AGE", "FARE", "SURVIVED").to_pandas()
    X = df[["EMBARKED", "SEX", "PCLASS", "AGE", "FARE"]]
    y = df["SURVIVED"]
    model = train(X, y)
    
    create_udf(model, session)
    return 'SUCCESS'


In [33]:
session.clear_imports()
session.clear_packages()
session.add_packages('snowflake-snowpark-python','pandas', 'scikit-learn', 'cloudpickle==2.0.0')
train_titanic_sp = F.sproc(func=train_titanic,name="train_titanic", is_permanent = True, replace= True, stage_location = 'udf_stage', session=session)

Run the training Stored Procedure in Snowflake

In [34]:
train_titanic_sp()

'SUCCESS'

In [35]:
titanic_df.select(F.call_udf("survived", F.object_construct('*')).as_("predicted"), F.col("SURVIVED").as_("actual")).show()

--------------------------
|"PREDICTED"  |"ACTUAL"  |
--------------------------
|1.0          |1         |
|1.0          |1         |
|0.0          |0         |
|0.0          |0         |
|0.0          |0         |
|0.0          |1         |
|1.0          |1         |
|0.0          |0         |
|1.0          |1         |
|0.0          |0         |
--------------------------



In [ ]:
session.close()